# Querying and Plotting the Lasair Database (PyroSQL) with local WFAU Archives (Firethorn)

_____________________________________________________________




In [ ]:
import pyrosql
import firethorn
import settings
from astropy.table import Table, vstack
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.graph_objs as go
import numpy as np

init_notebook_mode() # Important


# Setup Connection to Lasair Database

In [ ]:
psql = pyrosql.PyroSQL(settings.DB_HOST, settings.DB_USER, settings.DB_PASS, "3306", "MySQL")

# Run query to get a few candidates

In [ ]:
lasair_table = psql.execute_sql_query("SELECT cone_id,name,ra,decl FROM watchlist_cones WHERE wl_id=35", "ztf")

# Get number of rows returned

In [ ]:
lasair_table.count()
for i in lasair_table:
    print (i)

# Get results table as Astropy.Table

In [ ]:
lasair_astropy_table = lasair_table.as_astropy()

# Show astropy (interactive) table in notebook 

In [ ]:
lasair_astropy_table.show_in_notebook()


# Plot Light Curve for object

In [ ]:

mjd_list = [(x-2400000.5) for x in lasair_astropy_table["jd"]]

# Create a trace
trace = go.Scatter(
    x = mjd_list,
    y = lasair_astropy_table["magpsf"]
)

data = [trace]

iplot(data, filename='basic-line')

# Scatter Plot of (ra,dec)

In [ ]:
#Large Datasets

N = 100000
trace = go.Scattergl(
    x = lasair_astropy_table["ra"],
    y = lasair_astropy_table["dec"],
    mode = 'markers',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)
data = [trace]
iplot(data, filename='compare_webgl')

# Setup the Firethorn.py client to query WFAU catalogs

In [ ]:
ft = firethorn.Firethorn(endpoint='http://tap.roe.ac.uk/firethorn')

# Check available Archives and select one

In [ ]:
ft.get_public_resource_names() 

In [ ]:
resource = ft.get_public_resource_by_name('WSA')

# Create Workspace & Import a Database

In [ ]:
# Create new workspace
my_workspace = ft.new_workspace("mySpace")

In [ ]:
# Import a Schema from the OSA workspace
my_workspace.add_schema(
    resource.get_schema_by_name(
        "WFCAMCAL08B"
    )
)

# Run a Query

In [ ]:
# Run a Synchronous query
# Try these sources 858993625957,858993549466,858993615087,858993637017,858993759856)

query_text = """
SELECT b.sourceID,l.meanMjdObs,
ss.ra, ss.dec, 
ss.zaperMag3,ss.zaperMag3Err,
ss.yaperMag3,ss.yaperMag3Err,
ss.japerMag3,ss.japerMag3Err,
ss.haperMag3,ss.haperMag3Err,
ss.kaperMag3,ss.kaperMag3Err
FROM calSourceXSynopticSourceBestMatch AS b,calSynopticSource AS ss,calSynopticMergeLog AS l
WHERE b.synframeSetID=ss.synframeSetID AND b.synSeqNum=ss.synSeqNum AND b.synFrameSetID=l.synFrameSetID AND b.sourceID IN
(858993625957) 
AND japerMag3>0
"""

query = my_workspace.query(query_text)


In [ ]:
print(query.status())
if (query.status()=="FAILED"):
    print (query.error())

# Get and display as Astropy.Table

In [ ]:
# Get results table
ftpy_astropy_table = []

if (query.status()=="COMPLETED"):
    ftpy_table = query.results()
    
    # Get results table row count
    ftpy_table.rowcount()
    ftpy_astropy_table = ftpy_table.as_astropy()
    ftpy_astropy_table.show_in_notebook()

In [ ]:
ftpy_astropy_table.show_in_notebook()

In [ ]:
# Create a trace
trace = go.Scatter(
    x = ftpy_astropy_table["meanMjdObs"],
    y = ftpy_astropy_table["japerMag3"]
)

data = [trace]

iplot(data, filename='basic-line')

# Scatter Plot of (ra,dec)


In [ ]:

#Large Datasets

N = 100000
trace = go.Scattergl(
    x = ftpy_astropy_table["ra"],
    y = ftpy_astropy_table["dec"],
    mode = 'markers',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)
data = [trace]
iplot(data, filename='compare_webgl')

# Group the two Astropy Tables

In [ ]:
grouped_table = vstack([lasair_astropy_table, ftpy_astropy_table])

In [ ]:
print (grouped_table)

# Scatter Plot of Grouped Table

In [ ]:
#Large Datasets

N = 100000
trace = go.Scattergl(
    x = grouped_table["ra"],
    y = grouped_table["dec"],
    mode = 'markers',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)
data = [trace]
iplot(data, filename='compare_webgl')

# Initialise Aladin Lite client and Visualise data points

In [ ]:
import ipyaladin.aladin_widget as ipyal

aladin= ipyal.Aladin()
aladin



In [ ]:
aladin.add_table(lasair_astropy_table)

In [ ]:
aladin.add_table(ftpy_astropy_table)